In [2]:
import ccxt
import pandas as pd
import time
from datetime import datetime

In [3]:
def calcola_rendimento_portafoglio_ideale(colonna_prezzi, capitale_iniziale, bitcoin_iniziali, min_acquisto=0, min_vendita=0, perc_commissione_acquisto=0, perc_commissione_vendita=0):
    n = len(colonna_prezzi)
    
    # Tabella per memorizzare i massimi rendimenti
    dp = [[0] * 2 for _ in range(n)]  # dp[i][0]: massimo rendimento senza bitcoin, dp[i][1]: massimo rendimento con bitcoin

    # Stato iniziale
    dp[0][0] = capitale_iniziale  # Liquidità posseduta
    dp[0][1] = bitcoin_iniziali # Quantità bitcoin posseduti

    for i in range(1, n):
        prezzo_corrente = colonna_prezzi.iloc[i]

        # Caso 1: non ho bitcoin (capitale in liquidità)
        dp[i][0] = dp[i - 1][0]  # Mantengo il capitale
        if dp[i - 1][1] * prezzo_corrente >= min_vendita:  # Posso vendere bitcoin
            capitale_dopo_vendita = dp[i - 1][1] * prezzo_corrente * (1 - perc_commissione_vendita)
            dp[i][0] = max(dp[i][0], capitale_dopo_vendita)  # Massimo rendimento tra mantenere e vendere

        # Caso 2: ho bitcoin
        dp[i][1] = dp[i - 1][1]  # Mantengo i bitcoin
        if dp[i - 1][0] >= min_acquisto:  # Posso comprare bitcoin
            bitcoin_acquistati = (dp[i - 1][0] * (1 - perc_commissione_acquisto)) / prezzo_corrente
            dp[i][1] = max(dp[i][1], bitcoin_acquistati)  # Massimo rendimento tra mantenere e comprare

    # Rendimento finale: il massimo tra avere tutto in capitale o tutto in bitcoin all'ultimo prezzo
    valore_finale_capitale = dp[-1][0]
    valore_finale_bitcoin = dp[-1][1] * colonna_prezzi.iloc[-1] * (1 - perc_commissione_vendita)

    valore_iniziale = capitale_iniziale + bitcoin_iniziali * colonna_prezzi.iloc[0]
    valore_finale = max(valore_finale_capitale, valore_finale_bitcoin)

    rendimento = valore_finale / valore_iniziale

    return [rendimento, valore_finale]

In [4]:
def calcola_rendimento_portafoglio_ideale_multicrypto(lista_colonne_prezzi, capitale_iniziale, lista_token_iniziali, min_acquisto=0, min_vendita=0, perc_commissione_acquisto=0, perc_commissione_vendita=0):
    # lista_colonne_prezzi è una lista di pd.Series dei prezzi di ciascuna delle N crypto
    # Supponiamo: N = len(lista_colonne_prezzi)
    # Tutte le serie hanno la stessa lunghezza
    # i-esimo giorno: lista_colonne_prezzi[j][i] è il prezzo della j-esima crypto
    N = len(lista_colonne_prezzi)
    n = len(lista_colonne_prezzi[0])
    
    # Converto i prezzi in una matrice per comodità
    prezzi = []
    for i in range(n):
        prezzi_giorno = [lista_colonne_prezzi[j].iloc[i] for j in range(N)]
        prezzi.append(prezzi_giorno)

    # dp[i][0]: max capitale in USD se a fine giorno i non detengo crypto
    # dp[i][j]: max numero di token della j-esima crypto se a fine giorno i detengo crypto j (j = 1,...,N)
    # Nota: l'indice 0 in dp si riferisce allo stato "nessuna crypto"
    dp = [[float('-inf')] * (N+1) for _ in range(n)]

    # Stato iniziale
    dp[0][0] = capitale_iniziale
    for j in range(1, N+1):
        dp[0][j] = lista_token_iniziali[j-1]  # numero di token iniziali della j-esima crypto

    # Valore iniziale del portafoglio
    valore_iniziale = capitale_iniziale
    for j in range(N):
        valore_iniziale += lista_token_iniziali[j] * prezzi[0][j]

    for i in range(1, n):
        for stato_precedente in range(N+1):
            if dp[i-1][stato_precedente] == float('-inf'):
                continue
            
            # Caso 1: mantengo lo stesso stato (nessuna operazione)
            if stato_precedente == 0:
                # Nessuna crypto ieri, nessuna oggi
                dp[i][0] = max(dp[i][0], dp[i-1][0])
            else:
                # Avevo la crypto stato_precedente
                dp[i][stato_precedente] = max(dp[i][stato_precedente], dp[i-1][stato_precedente])

            # Caso 2: se ero senza crypto ieri, posso comprare crypto k oggi
            if stato_precedente == 0:
                # ho capitale in USD = dp[i-1][0]
                capitale = dp[i-1][0]
                for k in range(1, N+1):
                    if capitale >= min_acquisto:
                        # Compro crypto k
                        bitcoin_acquistati = (capitale * (1 - perc_commissione_acquisto)) / prezzi[i][k-1]
                        if bitcoin_acquistati * prezzi[i][k-1] >= min_acquisto:
                            dp[i][k] = max(dp[i][k], bitcoin_acquistati)

            # Caso 3: se avevo la crypto j ieri, posso vendere oggi e andare in USD
            if stato_precedente > 0:
                # stato_precedente è una crypto j-esima
                j_crypto = stato_precedente
                quantita = dp[i-1][j_crypto]  # numero di token
                valore_vendita = quantita * prezzi[i][j_crypto-1]
                if valore_vendita >= min_vendita:
                    capitale_dopo_vendita = valore_vendita * (1 - perc_commissione_vendita)
                    dp[i][0] = max(dp[i][0], capitale_dopo_vendita)

                # Caso 4: se avevo la crypto j, posso vendere e comprare un'altra crypto k
                for k in range(1, N+1):
                    if k != j_crypto:
                        # Vendo j
                        if valore_vendita >= min_vendita:
                            capitale_intermedio = valore_vendita * (1 - perc_commissione_vendita)
                            # Compro k
                            if capitale_intermedio >= min_acquisto:
                                bitcoin_acquistati = (capitale_intermedio * (1 - perc_commissione_acquisto)) / prezzi[i][k-1]
                                if bitcoin_acquistati * prezzi[i][k-1] >= min_acquisto:
                                    dp[i][k] = max(dp[i][k], bitcoin_acquistati)

    # Calcolo del valore finale
    valore_finale = dp[n-1][0]
    for j in range(1, N+1):
        if dp[n-1][j] != float('-inf'):
            # Vendo l'ultima crypto
            valore_uscita = dp[n-1][j] * prezzi[-1][j-1] * (1 - perc_commissione_vendita)
            valore_finale = max(valore_finale, valore_uscita)

    rendimento = valore_finale / valore_iniziale
    return [rendimento, valore_finale]

In [5]:
def fetch_crypto_data(market, timeframe, symbols, start_date, end_date):
    # Configura l'exchange
    try:
        exchange = getattr(ccxt, market)()
        exchange.load_markets()
        print(f"[INFO] Collegato all'exchange {market}.")
    except Exception as e:
        print(f"[ERROR] Errore nella connessione all'exchange {market}: {e}")
        return

    # Trasforma le date in timestamp
    try:
        start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
        end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)
        print(f"[INFO] Periodo scelto: da {start_date} a {end_date}.")
    except Exception as e:
        print(f"[ERROR] Errore nella conversione delle date: {e}")
        return

    # Lista per i dati scaricati
    all_data = {}

    for symbol in symbols:
        pair = f"{symbol}/USDT"
        if pair not in exchange.symbols:
            print(f"[WARNING] {pair} non è disponibile su {market}.")
            continue

        print(f"[INFO] Inizio il download dei dati per {pair} con timeframe {timeframe}.")

        ohlcv = []
        since = start_timestamp

        # Scarica i dati in blocchi
        while since < end_timestamp:
            try:
                batch = exchange.fetch_ohlcv(pair, timeframe, since)
                if not batch:
                    print(f"[INFO] Nessun dato aggiuntivo trovato per {pair}.")
                    break
                ohlcv.extend(batch)
                since = batch[-1][0] + 1  # Avanza al prossimo blocco
                print(f"[INFO] Scaricati {len(batch)} record per {pair} (fino a {datetime.utcfromtimestamp(batch[-1][0] / 1000)}).")
                time.sleep(exchange.rateLimit / 1000)  # Rispetta il rate limit
            except Exception as e:
                print(f"[ERROR] Errore durante il download dei dati per {pair}: {e}")
                break

        if not ohlcv:
            print(f"[WARNING] Nessun dato trovato per {pair}.")
            continue

        # Crea un DataFrame dai dati
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        all_data[pair] = df
        print(f"[INFO] Dati per {pair} scaricati con successo.")

    print("[INFO] Download completato.")
    return all_data

In [6]:
# Esempio di utilizzo
market_name = "bybit"  # Exchange
timeframe_download = '1h'  # Timeframe (ad esempio: '1m', '1h', '1d')
symbols_list = ['BTC', 'ETH', 'RENDER']  # Lista delle criptovalute
start_date = '2024-10-01'  # Data di inizio
end_date = '2024-11-25'  # Data di fine

crypto_data = fetch_crypto_data(market_name, timeframe_download, symbols_list, start_date, end_date)
crypto_data

# Salva i dati in file CSV
# if crypto_data:
#     for pair, df in crypto_data.items():
        # filename = f"{pair.replace('/', '_')}_{timeframe_download}_{start_date}_to_{end_date}.csv"
        # df.to_csv(filename)
        # print(f"[INFO] Dati per {pair} salvati in {filename}.")

[INFO] Collegato all'exchange bybit.
[INFO] Periodo scelto: da 2024-10-01 a 2024-11-25.
[INFO] Inizio il download dei dati per BTC/USDT con timeframe 1h.
[INFO] Scaricati 200 record per BTC/USDT (fino a 2024-10-09 07:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-10-17 14:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-10-25 21:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-11-03 04:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-11-11 11:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-11-19 18:00:00).
[INFO] Scaricati 199 record per BTC/USDT (fino a 2024-11-28 01:00:00).
[INFO] Dati per BTC/USDT scaricati con successo.
[INFO] Inizio il download dei dati per ETH/USDT con timeframe 1h.
[INFO] Scaricati 200 record per ETH/USDT (fino a 2024-10-09 07:00:00).
[INFO] Scaricati 199 record per ETH/USDT (fino a 2024-10-17 14:00:00).
[INFO] Scaricati 199 record per ETH/USDT (fino a 2024-10-25 21:00:00).
[INFO] Scaricati 199 

{'BTC/USDT':                          open      high       low     close       volume
 timestamp                                                               
 2024-10-01 00:00:00  63309.05  63598.30  62975.00  63526.86  2396.267596
 2024-10-01 01:00:00  63526.86  63639.64  63402.05  63457.17  1318.360698
 2024-10-01 02:00:00  63457.17  63457.18  63168.61  63438.23  1219.031439
 2024-10-01 03:00:00  63438.23  63744.86  63420.09  63715.97   868.203974
 2024-10-01 04:00:00  63715.97  63859.29  63641.27  63857.40   997.046908
 ...                       ...       ...       ...       ...          ...
 2024-11-27 21:00:00  96485.04  96582.73  96043.42  96264.15  1165.299606
 2024-11-27 22:00:00  96264.15  96516.18  96021.04  96037.57   624.364368
 2024-11-27 23:00:00  96037.57  96174.96  95719.36  95840.17   867.250239
 2024-11-28 00:00:00  95840.17  96561.44  95711.55  96541.37  1233.345009
 2024-11-28 01:00:00  96541.37  96550.73  96075.95  96267.34  1031.711344
 
 [1394 rows x 5 columns]

In [21]:
calcola_rendimento_portafoglio_ideale(crypto_data['BTC/USDT']['close'], 1000, 0, 5, 5, 0.001, 0.001)

[np.float64(7.94062532752397), np.float64(7940.625327523971)]

In [25]:
calcola_rendimento_portafoglio_ideale_multicrypto([crypto_data['BTC/USDT']['close'], crypto_data['ETH/USDT']['close']], 1000, [0, 0], 5, 5, 0.001, 0.001)

[np.float64(25.020366067580856), np.float64(25020.366067580857)]